# <center>UGCA Project 2</center>
## <center>Sentiment Analysis of the 2018 Texas Senate Race</center>

<center><font size=2>Yuejia Feng, Ting-Yi Hung, Abhilasha Kanitkar, and Brian Peterson</font><center>
    <center><font size=2>10.21.18</font></center>

## Task A

### Parse Tweets

In [28]:
import json
from dict_tools import find_key_in_dict

In [29]:
with open('ConsolidatedTweetsData.json','r') as f:
#     print(sum([1 for i in f]))
    count = 1
    raw_tweets = dict()
    for line in f:
        tweet = json.loads(line)
        tweet_specifics = find_key_in_dict(['full_text', 'location', 'text'], tweet)
        d = {}
        d['location'] = tweet_specifics['location']
        if 'full_text' in tweet_specifics.keys():
            d['text'] = tweet_specifics['full_text']
        else:
            d['text'] = tweet_specifics['text']
        raw_tweets[count] = d
        count += 1

In [30]:
raw_tweets

{1: {'location': 'Florida, USA',
  'text': '"Ted Cruz has insisted that we repeal every single word of the Affordable Care Act, including protections for pre-existing conditions. I want to expand medicaid."\n\n— Beto O\'Rourke talks differences between himself and Ted Cruz in the Texas Senate race. https://t.co/Ko7Y8ttM2I'},
 2: {'location': 'New York, N.Y.',
  'text': 'What the hell is wrong with Texas? Republicans have one of the smarmiest men ever as their candidate for Senate, Ted Cruz. Democrats have one of the best candidates in a lifetime, Beto O’Rourke. And yet Cruz is up by eight points.\n#Beto4Texas'},
 3: {'location': 'El Paso, TX',
  'text': '🛑Beto skates across the Stage on his skateboard. Hoping to show his “Cool Guy” side.  Cruz still leads in polling.Dont become complacent #Texas He’s raised $36 million to win. @realDonaldTrump https://t.co/dzlvM7GpIK🛑 https://t.co/MQQhmi076T'},
 4: {'location': None,
  'text': '"Ted Cruz has insisted that we repeal every single word of

In [31]:
len(raw_tweets)

7257

### Clean Tweets

In [32]:
import copy
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import brown
stopwords_set = set(stopwords.words('english'))
brown_set = set(brown.words())

In [33]:
tweets_cleaned = copy.deepcopy(raw_tweets)

In [34]:
characters_to_clean = r'?!,:,/\"-+=@#$%^&*()><{}[]|.' + r"'"

for tweet in tweets_cleaned:
    
    # clean the text
    for char in characters_to_clean:
        tweets_cleaned[tweet]['text'] = tweets_cleaned[tweet]['text'].replace(char, '')

    words = word_tokenize(tweets_cleaned[tweet]['text'].lower())
    cleaned_words = [word for word in words if word not in stopwords_set]
    cleaned_words1 = [word for word in cleaned_words if len(word)>2]
    
    tweets_cleaned[tweet]['text'] = ' '.join(cleaned_words1)
    
    #clean the location
    if tweets_cleaned[tweet]['location']:
        for char in characters_to_clean:
            tweets_cleaned[tweet]['location'] = tweets_cleaned[tweet]['location'].replace(char, '')
        tweets_cleaned[tweet]['location'] = tweets_cleaned[tweet]['location'].lower()        

In [35]:
tweets_cleaned

{1: {'location': 'florida usa',
  'text': 'ted cruz insisted repeal every single word affordable care act including protections preexisting conditions want expand medicaid beto orourke talks differences ted cruz texas senate race httpstcoko7y8ttm2i'},
 2: {'location': 'new york ny',
  'text': 'hell wrong texas republicans one smarmiest men ever candidate senate ted cruz democrats one best candidates lifetime beto rourke yet cruz eight points beto4texas'},
 3: {'location': 'el paso tx',
  'text': '🛑beto skates across stage skateboard hoping show cool guy side cruz still leads pollingdont become complacent texas raised million win realdonaldtrump httpstcodzlvm7gpik🛑 httpstcomqqhmi076t'},
 4: {'location': None,
  'text': 'ted cruz insisted repeal every single word affordable care act including protections preexisting conditions want expand medicaid beto orourke talks differences ted cruz texas senate race httpstcoko7y8ttm2i'},
 5: {'location': 'little blue speck of dust',
  'text': 'hell 

## Task B

### Tweet Statistics

In [36]:
from collections import Counter
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [37]:
cnt = Counter()
for i in tweets_cleaned:
    cnt += Counter(word_tokenize(tweets_cleaned[i]['text']))

In [38]:
cnt.most_common(1000)

[('endorsed', 6623),
 ('texas', 6408),
 ('senate', 5896),
 ('beto', 5873),
 ('orourke', 4684),
 ('houston', 4448),
 ('chronicle', 4435),
 ('cruz', 4241),
 ('ted', 3827),
 ('breaking', 3561),
 ('betoorourke', 3267),
 ('president', 3127),
 ('vote', 3043),
 ('obama', 2692),
 ('newspaper', 2673),
 ('bush', 2673),
 ('george', 2666),
 ('romney', 2663),
 ('ran', 2649),
 ('httpstcobbzimtil8z', 2644),
 ('race', 1902),
 ('krassenstein', 908),
 ('would', 890),
 ('one', 771),
 ('endorses', 685),
 ('texans', 605),
 ('republicans', 596),
 ('thehill', 516),
 ('trump', 511),
 ('says', 420),
 ('candidates', 415),
 ('years', 400),
 ('candidate', 396),
 ('rourke', 396),
 ('good', 386),
 ('httpstco2yx0ugcqec', 383),
 ('httpstcokfqhhmwwro', 383),
 ('teambeto', 377),
 ('say', 366),
 ('country', 357),
 ('realdonaldtrump', 352),
 ('endorsing', 346),
 ('yes', 344),
 ('impeach', 344),
 ('wrong', 342),
 ('dem', 342),
 ('utterly', 339),
 ('willing', 336),
 ('federal', 332),
 ('impeachment', 332),
 ('madness', 331

# <font color='red'> Once all tweets are collected, use above function to find issues </font>

In [ ]:
#Issues found 
#medicaid, people, security, terrorists, border, danger , protections ,attacks, education ,money, radical, thug, nonwhites, corrupt, attacks, healthcare, illegal, immigration, racist

## Task C

### Find and count tweets mentioning each issue

In [39]:
issues = {
    'healthcare': dict(),
    'job': dict(),
    'crime': dict(),
    'gun': dict(),
    'education' : dict(),
    'immigration' : dict()
    
    }

In [40]:
for i in tweets_cleaned:
    for j in word_tokenize(tweets_cleaned[i]['text']):
        if j == 'medicaid' or j == 'health' or j=='healthcare':
            issues['healthcare'][i] = tweets_cleaned[i]['text']
        elif j == 'job' or j == 'jobs':
            issues['job'][i] = tweets_cleaned[i]['text']
        elif j=='education':
            issues['education'][i] = tweets_cleaned[i]['text']
        elif j == 'crime' or j == 'crimes':
            issues['crime'][i] = tweets_cleaned[i]['text']
        elif j == 'gun' or j == 'guns':
            issues['gun'][i] = tweets_cleaned[i]['text']
        elif j=='immigration':
            issues['immigration'][i] = tweets_cleaned[i]['text']

#pp.pprint(issues['education'].values())

In [41]:
issues_mention_list = []
for i in issues:
    issues_mention_list.append( (i, len(issues[i])))
issues_mention_list.sort(key=lambda x: x[1], reverse=True)
# print(len(issues_mention_list))
for i in range(len(issues_mention_list)):
    print(str(i+1) + ") The issue '" + str(issues_mention_list[i][0],) + "' was mentioned " + str(issues_mention_list[i][1]) + ' times')
      

1) The issue 'healthcare' was mentioned 106 times
2) The issue 'job' was mentioned 44 times
3) The issue 'education' was mentioned 14 times
4) The issue 'immigration' was mentioned 7 times
5) The issue 'gun' was mentioned 3 times
6) The issue 'crime' was mentioned 1 times


### Find and count tweets mentioning each candidate 

In [42]:
candidates = {'beto':dict(), 'cruz':dict()}

for i in tweets_cleaned:
    for j in word_tokenize(tweets_cleaned[i]['text']):
        if j == 'beto' or j == "o'rourke" or j == "o'rourke" or j == "betoorourke":
            candidates['beto'][i] = tweets_cleaned[i]['text']
        elif j == 'cruz' or j == 'ted':
            candidates['cruz'][i] = tweets_cleaned[i]['text']

#pp.pprint(candidates)

In [43]:
candidates_mention_list = []
for i in candidates:
    candidates_mention_list.append( (i, len(candidates[i])))
candidates_mention_list.sort(key=lambda x: x[1], reverse=True)
# print(len(issues_mention_list))
for i in range(len(candidates_mention_list)):
    print(str(i+1) + ") The candidates '" + str(candidates_mention_list[i][0],) + "' was mentioned " + str(candidates_mention_list[i][1]) + ' times')
    

1) The candidates 'beto' was mentioned 6296 times
2) The candidates 'cruz' was mentioned 3957 times


### Find cross mentions

In [44]:
# init dictionary
cross_candidates_issues = dict()
for i in issues:
    cross_candidates_issues[i] = dict()
    for j in candidates:
        cross_candidates_issues[i][j] = []
pp.pprint(cross_candidates_issues)
        

{ 'crime': {'beto': [], 'cruz': []},
  'education': {'beto': [], 'cruz': []},
  'gun': {'beto': [], 'cruz': []},
  'healthcare': {'beto': [], 'cruz': []},
  'immigration': {'beto': [], 'cruz': []},
  'job': {'beto': [], 'cruz': []}}


In [45]:
for i in candidates:  #for every candidate
    for j in issues:  # then for every issue
        for k in issues[j].values():
            for l in word_tokenize(k):
                if l == i:
                    cross_candidates_issues[j][i].append(k) 
                    break
pp.pprint(cross_candidates_issues)

{ 'crime': {'beto': [], 'cruz': []},
  'education': { 'beto': [ 'broumleyfc tedcruz betoorourke still dont get beto '
                           'like texas education thattexas running senate '
                           'federal education stuff affect like texas '
                           'vetobeto'],
                 'cruz': []},
  'gun': { 'beto': [ 'betoorourke repjoekennedy call gun violence survivor '
                     'texas momsdemand volunteer diana earl spiceygirl512 road '
                     'trip diana driven across state beto rallies since '
                     'february went five one day 💪💪💪 httpstcoxrkhxzju3s',
                     'ted cruz makes texas great elect liberal democrat beto '
                     'orourke hell try take guns away amp prevent happening '
                     'beto betoorourke betofortexas betomale tedcruz cruz '
                     'texasshooting wellregulatedmilitia goodguyswithguns'],
           'cruz': [ 'ted cruz makes texas great

### Calculate Lift

In [46]:
# now we can calculate the lift value
data=[]
for i in range(len(cross_candidates_issues.keys())):
    row = []
    for j in range(len(cross_candidates_issues[list(cross_candidates_issues.keys())[i]].keys())):
            row.append(round( len(cross_candidates_issues[list(issues.keys())[i]][list(candidates.keys())[j]])\
                             / (len(issues[list(issues.keys())[i]])\
                            * len(candidates[list(candidates.keys())[j]]) ) * len(tweets_cleaned), 4))
    data.append(row)
data

[[1.033, 1.6783],
 [0.8645, 1.2087],
 [0.0, 0.0],
 [0.7684, 0.6113],
 [0.0823, 0.0],
 [0.0, 0.262]]

In [54]:
import pandas as pd
import numpy as np
lift_computed = []
for i in data:
    for j in i:
        lift_computed.append(j)
lift_computed
indexes = []
for i in issues.keys():
    for j in candidates.keys():
        a = str(i+' & '+j)
        indexes.append(a)
indexes
lift_chart = pd.DataFrame(lift_computed, columns=["Lift"], index=indexes)
lift_chart

,Lift
healthcare & beto,1.0330
healthcare & cruz,1.6783
job & beto,0.8645
job & cruz,1.2087
crime & beto,0.0000
crime & cruz,0.0000
gun & beto,0.7684
gun & cruz,0.6113
education & beto,0.0823
education & cruz,0.0000


### Calculate Sentiment

In [55]:
# calculating sentiment for only those where lift >1
issues1 = []
candidates1 = []
for i in range(len(cross_candidates_issues.keys())):
    for j in range(len(cross_candidates_issues[list(cross_candidates_issues.keys())[i]].keys())):
        lift = round( len(cross_candidates_issues[list(issues.keys())[i]][list(candidates.keys())[j]])\
                    / (len(issues[list(issues.keys())[i]])\
                    * len(candidates[list(candidates.keys())[j]]) ) * len(tweets_cleaned), 4)
        if lift > 1:
            issues1.append(list(issues.keys())[i])
            candidates1.append(list(candidates.keys())[j])
    
issues1,candidates1

(['healthcare', 'healthcare', 'job'], ['beto', 'cruz', 'cruz'])

In [61]:
cross_lift_high1 = {}
# init dictionary for storing 'lift > 1' of cross_candidates_issues only.
for c in range(len(issues1)):
    for i in issues:
        if i in cross_lift_high1.keys():
            for j in candidates:
                if i == issues1[c] and j == candidates1[c]:
                    cross_lift_high1[i][j]= []
        elif i == issues1[c]:
            cross_lift_high1[i]= dict()
            for j in candidates:
                if i == issues1[c] and j == candidates1[c]:
                    cross_lift_high1[i][j]= []
print(cross_lift_high1)

count = 0
for c in range(len(issues1)):
    for i in cross_candidates_issues: #i  healthcare
        for j in cross_candidates_issues[i]: #j  beto
            if i == issues1[c] and j == candidates1[c]:
                for k in cross_candidates_issues[i][j]:
                    cross_lift_high1[i][j].append(k)
                
#cross_lift_high1

{'healthcare': {'beto': [], 'cruz': []}, 'job': {'cruz': []}}


In [57]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def cal_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    return snt['compound']

In [58]:
from nltk import ngrams
def get_radius_list(sentence,str1):
#     sentence = 'new poll finds beto trailing 5but leading cruz by 60 points among12 voters who say health care is their top issue this is why beto 014has gone all in on xxdefendingxx obamacare and calling for medicaid expansion'
    long = len(sentence.split())
    # limit = int(((long-n)/n)+n)
    n = 13
    limit = long-(n-1) # word long = 37 , pick 12 words every time. 25th time is the last time.
    
    sixgrams = ngrams(sentence.split(), n)
    count= 1
    temp = []
#     str1 = 'expansion'

    for grams in sixgrams:
        if count == 1:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 2:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 3:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 4:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 5:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 6:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 7:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count < limit:
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == limit:
            for i in range(6,13):
                if grams[i] == str1:
                    c = ' '.join(grams)
                    temp.append(c)
        count += 1
    return temp

In [59]:
cross_lift_score = dict()

# init dictionary to store the Sentiment Score
for i in issues:
    cross_lift_score[i] = dict()
    for j in candidates:
        cross_lift_score[i][j] = None 
print(cross_lift_score)

{'healthcare': {'beto': None, 'cruz': None}, 'job': {'beto': None, 'cruz': None}, 'crime': {'beto': None, 'cruz': None}, 'gun': {'beto': None, 'cruz': None}, 'education': {'beto': None, 'cruz': None}, 'immigration': {'beto': None, 'cruz': None}}


In [62]:
total = 0
for i in cross_lift_high1: #healthcare
    if i == 'healthcare':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(k):
#                 for w in word_tokenize(cross_lift_high1[i][j][k]):

                    if w == 'medicaid' or w == 'health' or w == 'healthcare':
                        for l in get_radius_list(k,w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
    if i == 'job':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(k):
                    if w == 'job' or w == 'jobs':
                        for l in get_radius_list(k,w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
    if i == 'education':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(k):
                    if w == 'education':
                        for l in get_radius_list(k,w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
    
    if i == 'crime':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(k):
                    if w == 'crime' or w == 'crimes':
                        for l in get_radius_list(k,w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
    if i == 'gun':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(k):
                    if w == 'gun' or w == 'guns':
                        for l in get_radius_list(k,w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
    if i == 'immigration':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(k):
                    if w == 'immigration':
                        for l in get_radius_list(k,w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
cross_lift_score

{'healthcare': {'beto': 0.39067368421052545, 'cruz': 0.776391752577317},
 'job': {'beto': None, 'cruz': 2.5098999999999925},
 'crime': {'beto': None, 'cruz': None},
 'gun': {'beto': None, 'cruz': None},
 'education': {'beto': None, 'cruz': None},
 'immigration': {'beto': None, 'cruz': None}}

In [63]:
# Extracting the score from dictionary into list.
total_score = []
for i in cross_lift_score:
    for j in cross_lift_score[i]:
        total_score.append(cross_lift_score[i][j])
total_score

[0.39067368421052545,
 0.776391752577317,
 None,
 2.5098999999999925,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [64]:
indexes = []
for i in issues.keys():
    for j in candidates.keys():
#         print(str(i+'&'+j))
        a = str(i+' & '+j)
        indexes.append(a)
indexes
total_score_df = pd.DataFrame({'Sentiment Score (if lift > 1)':total_score}, index = indexes )
total_score_df

,Sentiment Score (if lift > 1)
healthcare & beto,0.390674
healthcare & cruz,0.776392
job & beto,NaN
job & cruz,2.509900
crime & beto,NaN
crime & cruz,NaN
gun & beto,NaN
gun & cruz,NaN
education & beto,NaN
education & cruz,NaN


In [29]:
final_chart = pd.concat([lift_chart,total_score_df],axis=1)
final_chart

,Lift,Sentiment Score (if lift > 1)
healthcare & beto,1.7291,1.224800
healthcare & cruz,0.0000,NaN
job & beto,0.5895,NaN
job & cruz,1.3138,-0.046361
crime & beto,0.0000,NaN
crime & cruz,0.0000,NaN
gun & beto,1.7291,-1.968600
gun & cruz,0.6882,NaN


## Task D

### MDS

## Task E

### Lift Values

In [65]:
def find_key_in_dict(keys_i_want, dictionary):
    l = []
    def iterate_through_dict(keys_i_want, dictionary):
        for i in dictionary:
            if i in keys_i_want:
                l.append({i: dictionary[i]})
                keys_i_want.remove(i)
            if type(dictionary[i]) is dict:
                iterate_through_dict(keys_i_want, dictionary[i])
    iterate_through_dict(keys_i_want,dictionary)            
    return l

In [66]:
with open('ConsolidatedTweetsData.json','r') as f:
#     print(sum([1 for i in f]))
    comments_list =[]
    #big_city = ['Dallas', 'Houston', 'San Antonio', 'Austin', 'El Paso']
    count = 1
    for line in f:
        tweet = json.loads(line)
        if count <= 4000:
            l = find_key_in_dict(['full_text', 'location', 'text'], tweet)
            d = {}
            for i in range(len(l)):
                for j in l[i]:
                    if j == 'location':
                        d['location'] = l[i][j]
                        break
                for j in l[i]:
                    if j == 'full_text':
                        d['text'] = l[i][j]
                        break
                if 'text' not in d.keys():
                    for j in l[i]:
                        if j == 'text':
                            d['text'] = l[i][j]
                            break
            comments_list.append(d)
#             print(count,d)
#             print()
        count += 1
    
comments_list

[{'text': '"Ted Cruz has insisted that we repeal every single word of the Affordable Care Act, including protections for pre-existing conditions. I want to expand medicaid."\n\n— Beto O\'Rourke talks differences between himself and Ted Cruz in the Texas Senate race. https://t.co/Ko7Y8ttM2I',
  'location': 'Florida, USA'},
 {'text': 'What the hell is wrong with Texas? Republicans have one of the smarmiest men ever as their candidate for Senate, Ted Cruz. Democrats have one of the best candidates in a lifetime, Beto O’Rourke. And yet Cruz is up by eight points.\n#Beto4Texas',
  'location': 'New York, N.Y.'},
 {'text': '🛑Beto skates across the Stage on his skateboard. Hoping to show his “Cool Guy” side.  Cruz still leads in polling.Dont become complacent #Texas He’s raised $36 million to win. @realDonaldTrump https://t.co/dzlvM7GpIK🛑 https://t.co/MQQhmi076T',
  'location': 'El Paso, TX'},
 {'text': '"Ted Cruz has insisted that we repeal every single word of the Affordable Care Act, includ

__Replace 10 biggest cities by population (e.g., Dallas, Houston, San Antonio, Austin, El Paso, etc.) with big_city and the rest with small_town. Then merge the location and the text columns. __

In [67]:
big_city = ['Dallas', 'Houston', 'San Antonio', 'Austin', 'El Paso','Fort Worth','Arlington','Corpus Christi','Plano','Laredo']
comments = []
for i in comments_list:
    m =str(i['text'])
    l = str(i['location']).split(",")[0]
    if l in big_city:
        l = "big_city"
    else:
         l = "small_town"
    comments.append(l+' '+m)
          

len(comments )


4000

In [68]:
comments=list(map(lambda x: x.lower(), comments))
comments

['small_town "ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i',
 'small_town what the hell is wrong with texas? republicans have one of the smarmiest men ever as their candidate for senate, ted cruz. democrats have one of the best candidates in a lifetime, beto o’rourke. and yet cruz is up by eight points.\n#beto4texas',
 'big_city 🛑beto skates across the stage on his skateboard. hoping to show his “cool guy” side.  cruz still leads in polling.dont become complacent #texas he’s raised $36 million to win. @realdonaldtrump https://t.co/dzlvm7gpik🛑 https://t.co/mqqhmi076t',
 'small_town "ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'ro

## 2. Calculate the lift value

In [70]:
# Number of each type of city mentioned in the tweets
city = {'big_city', 'small_town'}
city_mention = dict()

#initialize counter
for i in city:
    city_mention[i] = []
    
for i in city:
    for j in comments:
        for k in word_tokenize(j):
            if k == i:
                city_mention[i].append(k)
            break
    
    print(i + " was totally mentioned "+str(len(city_mention[i]))+" times.")

#print(city_mention)

big_city was totally mentioned 189 times.
small_town was totally mentioned 3811 times.


In [71]:
#Number of each name mentioned in the tweets
name = {'beto':['beto', 'betoorourke','@betoorourke'], 'cruz':['cruz','tedcruz','@tedcruz' ]}
name_mention = dict()


#initialize counter
for i in name:
    name_mention[i] = []
    
    
for i in name:
    for m in name[i]:
        for j in comments:
             for k in word_tokenize(j):
                     if k == m:
                            name_mention[i].append(j)
                            break                    
    print(i + " was totally mentioned "+str(len(name_mention[i]))+" times.")

beto was totally mentioned 4803 times.
cruz was totally mentioned 2486 times.


In [72]:
# Number of tweets containing each pair of city type and name
#(big_city, beto),(big_city, cruz),(small_town, beto),(small_town,, cruz)

# for each city type mentioned, find number of times each of name was also mentioned
cross_mention = dict()

# init dictionary
for i in city_mention:
    cross_mention[i] = dict()
    for j in name_mention:
        cross_mention[i][j] = [ ]
#{'big_city': {'beto': [], 'cruz': []}, 'small_town': {'beto': [], 'cruz': []}}

for i in city:
    for j in name_mention:
        for k in range(len(name_mention[j])):
            for l in word_tokenize(name_mention[j][k]):
                if l == i:
                    cross_mention[i][j].append( name_mention[j][k])
                    break
        print("Pair "+"("+i+","+j+")" + " was totally mentioned "+str(len(cross_mention[i][j]))+" times.")




Pair (big_city,beto) was totally mentioned 237 times.
Pair (big_city,cruz) was totally mentioned 118 times.
Pair (small_town,beto) was totally mentioned 4566 times.
Pair (small_town,cruz) was totally mentioned 2368 times.


In [73]:
N = len(comments)
N

4000

In [74]:
# now we can calculate the lift value
lift = [ ]

for i in range(len(city_mention.keys())):
    row = []
    for j in range(len(name_mention.keys())):
        row.append( round(N*len(cross_mention[list(city_mention.keys())[i]][list(name_mention.keys())[j]])/((len(city_mention[list(city_mention.keys())[i]]))*(len(name_mention[list(name_mention.keys())[j]]))), 4))

    lift.append(row)
lift

[[1.0443, 1.0046], [0.9978, 0.9998]]

In [75]:
lift_chart = pd.DataFrame(lift, columns=name_mention.keys(), index=city_mention.keys())
lift_chart

,beto,cruz
big_city,1.0443,1.0046
small_town,0.9978,0.9998


## 3. Calculate sentiment score

In [76]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [77]:
def cal_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    return snt['compound']

In [78]:
#calculate sentiment score
score = []

#For each combination of each city type and each name(beto or cruz), 
#firstly calculate the sentiment score of each sentence of the combination, then calculate the average of the scores
#as the final sentiment score ot this combination

for i in list(city_mention.keys()):
    row=[]
    for j in list(name_mention.keys()):
        a=0
        for k in cross_mention[i][j]:
             a+=cal_sentiment_scores(k)
        row.append(round(a/len(cross_mention[i][j]),4))
    score.append(row)
            
score
        

[[0.3589, 0.2451], [0.3323, 0.2806]]

In [79]:
sentiment_score_chart = pd.DataFrame(score, columns=name_mention.keys(), index=city_mention.keys())
sentiment_score_chart

,beto,cruz
big_city,0.3589,0.2451
small_town,0.3323,0.2806


## Task F

<h4>What advice would you give to each of the two candidates based on your analyses in C, D and E above?</h4>